In [2]:
import pandas as pd
import numpy as np
data  = pd.read_csv("./data/Drugs.csv")
data.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [5]:
r = pd.pivot_table(data,index=['State','COUNTY','SubstanceName']
                  ,columns = 'YYYY'
                  ,values='DrugReports',fill_value='-').reset_index().rename_axis(columns={'YYYY':''})

In [6]:
r.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


In [9]:
result_melted = r.melt(id_vars=r.columns[:3],value_vars=r.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY'
                                    ,'SubstanceName']).reset_index().drop(columns='index')
#下面其实无关紧要，只是交换两个列再改一下类型（因为‘-’所以type变成object了）
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols].astype({'DrugReports':'int','YYYY':'int'})
result2.head()

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2


In [11]:
df = pd.read_csv('./data/Earthquake.csv').head()
df.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [17]:
r = r = pd.pivot_table(df,index=['日期','时间','纬度','经度']
                  ,columns='方向'
                  ,values=['烈度','深度','距离'],fill_value='-').stack(level=0).rename_axis(index={None:'地震参数'})

In [18]:
r.head()

地震参数  地震参数
方向    深度       7.00
      烈度       0.00
      经度      30.24
      距离       0.10
日期    深度      10.00
dtype: float64

In [23]:
df_result = r.unstack().stack(0)[(~(r.unstack().stack(0)=='-')).any(1)].reset_index()
df_result.columns.name=None
df_result = df_result.sort_index(axis=1).astype({'深度':'float64','烈度':'float64','距离':'float64'})
df_result.head()

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>